## `model_checker` Python API Jupyter notebook
This jupyter notebook will go through the basics of how to use the `model_checker` package from scratch in python. It will also go over how to use some of the built-in features that make exploring semantics already in `theory_lib` much easier. 

First, the basic imports. If you want to make a theory from scratch, see the `semantic.py` and `operator.py` files for the theories in `theory_lib`. This notebook will use the semantics for Bernard and Champollion's exclusion semantics (which is what the second import is).

In [ ]:
import model_checker
from model_checker.theory_lib import exclusion
# from model_checker import exxclusion

First we need operators. We can define these, or, since we are using the exclusion semantics as an example, we can import them from a theory in theory_lib. Even with that, we have two options. We can import a group of default operators for that theory, or we can import specific operators from the operator file. In either case, operators are imported through a Syntactic.OperatorCollection object:

In [ ]:
# import the default operators of the theory:
operators = exclusion.exclusion_operators
assert isinstance(operators, model_checker.syntactic.OperatorCollection)

# import specific operators manually:
operators = model_checker.syntactic.OperatorCollection(exclusion.operators.ExclusionOperator, # Negation
                                            exclusion.operators.UniAndOperator, # And
                                            exclusion.operators.UniOrOperator, # Or
                                            exclusion.operators.UniIdentityOperator # Propositional Identity
                                            )

Since the default `OperatorCollection` object for the exclusion semantics contains exactly those operators seen in the manual import, the two `operators` objects above are equivalent (comparison for equality is not (yet?) supported for `OperatorCollection` objects). 

We now need to pick a theorem to test. The operators have names associated to them, and it is these that you type in the string that represents the theorem. You can see the names of these operators by printing them:

In [10]:
for operator in operators:
    print(operator) # hopefully it's clear which operators correspond to which! 

\exclude
\uniwedge
\univee
\uniequiv


Now we need to import the syntax. To do this, we need a theorem to get a model for. A `Syntax` object will represent the theorems inputted as strings in a way that `model_checker` can understand. It doesn't really have any attributes useful to the user. For more details, see the `syntactic.py` module of `model_checker`.

In [ ]:
premises = ["\\exclude (P \\uniwedge Q)"]
conclusions = ["(\\exclude P \\univee \\exclude Q)"]
syntax = model_checker.syntactic.Syntax(premises, conclusions, operators)

Now we make the semantics. We will just use the one in the `theory_lib` for exclusion semantics. A `Semantics` object, of which `ExclusionSemantics` is an instance, has limited user-useful attributes, which are shown below. For more details, see the `SemanticDefaults` class of the `model.py` module of `model_checker` and the `[X]Semantics` object found in the `semantic.py` module of the theory you are interested in. To initialize as a `Semantics` object, feed it the number of atomic states you want in your model. We represent this value with `N`. [UPDATE: This has changed. Now you feed it the settings you want the semantics to have. We will just use the theory's default settings for ease. ]

In [12]:
# N = 3 # number of atomics tates desired
default_settings = exclusion.ExclusionSemantics.DEFAULT_EXAMPLE_SETTINGS
print(default_settings)
semantics = exclusion.ExclusionSemantics(default_settings)
print(str(semantics))
print(semantics.N) # number of atomic states in the model
# print(semantics.DEFAULT_EXAMPLE_SETTINGS) # default settings. Will be useful in next step. 

{'N': 3, 'possible': False, 'contingent': False, 'non_empty': False, 'non_null': False, 'disjoint': False, 'fusion_closure': False, 'iterate': 1, 'max_time': 1, 'expectation': None}
3


The last ingredient we need before we get the ball rolling is a `Proposition` class. As with the semantics there is a default for the exclusion semantics in its module; for details on how to make one from the ground up, see the `PropositionDefaults` class in the `model.py` module of `model_checker` and the `[X]Proposition` object in the `semantic.py` modules of the theories in `theory_lib`. 

In [13]:
proposition_class = exclusion.UnilateralProposition

Now that we have a syntax with a specific example, a semantics, and a definition of a proposition, we can in principle find all the constraints that would be needed to see if a countermodel exists. And indeed this is what we do next, by defining a `ModelConstraints` object. Since we can impose additional constraints on the model (e.g., only contingent sentences, only non-empty verifiers, only non-null falsifiers), we also pass a `settings` dictionary into the `ModelConstraints` object. The `ExclusionSemantics` has a default setting dictionary, which was printed in the cell above; we will use this for now. 

A `Model_Constraints` object has many user-useful attributes, mainly in the form of Z3 constraints. Z3's constraints are in prefix notation; also for more details see the specific implementations of constraints in the `Semantics` object and `Proposition` object you have. 

In [ ]:
settings = semantics.DEFAULT_EXAMPLE_SETTINGS
model_constraints = model_checker.model.ModelConstraints(settings, syntax, semantics, proposition_class)

# print(model_constraints.frame_constraints)
# print(model_constraints.model_constraints)
# print(model_constraints.premise_constraints)
# print(model_constraints.conclusion_constraints)
print(model_constraints.all_constraints)

These may seem initially opaque, so it is recommended to see the specific implementations of constraints in the `Semantics` object and `Proposition` object you have. Additionally note that any quantifiers in definitions are finite—i.e. we do not use the default quantifiers from 'z3'. This is because this is much faster for small state spaces. 

Now we define the model structure. This will be user-defined (?), but mostly just for printing; the bulk of it is defined in the `ModelDefaults` class of the `model.py` module of `model_checker`. We also need to feed in some settings; we can use the settings from above as default in case we don't want anything interesting. 

Defining this object automatically solves the constraints and finds a model. (i.e. this is where the Z3 magic happens) We can print the `z3` model—which is sometimes useful for debugging—if a model was found or the `z3` `unsat_core` (see here for more details: https://z3prover.github.io/api/html/ml/Z3.Solver.html) if no model was found. We can also print the model in a user-friendly way, which is what the last step does. 

There are many user-useful print methods defined in the model structure, both in the theory-specific object (entire point of defining this object) and the default (in the position mentioned in the second paragraph). It is recommended to look at those. 

In [15]:
model_structure = exclusion.ExclusionStructure(model_constraints, settings)
# NOTE: below not working because 'expectation' key not in settings
# model_structure.print_model() # prints the z3 model or the unsat core, depending on if model found
model_structure.print_all(settings, "DeMorgan's", "Exclusion Semantics")


EXAMPLE DeMorgan's: there is no countermodel.

Atomic States: 3

Semantic Theory: Exclusion Semantics

Premise:
1. \exclude (P \uniwedge Q)

Conclusion:
2. (\exclude P \univee \exclude Q)

Z3 Run Time: 0.0185 seconds




No countermodels found, as desired! Now we will try a different theorem, for which we expect countermodels in the exclusion semantics. Namely, the propositional equivalence between the two sides of DeMorgan's tested above. The good thing is, we only need to redefined the `Syntax` object, `ModelConstraints` object, and the `ModelStructure` object—the semantics and proposition class are reusable from before, highlighting the convenient modularity of the `model_checker`! 

In [ ]:
premises2 = ["(\\exclude (P \\uniwedge Q) \\uniequiv (\\exclude P \\univee \\exclude Q))"]
conclusions2 = []

DEFAULT_GENERAL_SETTINGS = {
        "print_impossible": False,
        "print_constraints": False,
        "print_z3": False,
        "save_output": False,
        "maximize": False,
    }

for key in DEFAULT_GENERAL_SETTINGS:
    settings[key] = DEFAULT_GENERAL_SETTINGS[key]

syntax2 = model_checker.syntactic.Syntax(premises2, conclusions2, operators)
model_constraints2 = model_checker.model.ModelConstraints(settings, syntax2, semantics, proposition_class)
model_structure2 = exclusion.ExclusionStructure(model_constraints2, settings)

# This is where the fix is - interpret sentences before printing
sentences = model_structure2.premises + model_structure2.conclusions
model_structure2.interpret(sentences)

# Now print the results
model_structure2.print_all(settings, "DeMorgan's Identity", "Exclusion Semantics")